In [16]:
import elasticsearch_dsl
from elasticsearch import Elasticsearch

client = Elasticsearch(hosts=['10.0.3.33:9200'])

In [17]:
import uuid
from faker import Faker
import random
fake = Faker()

student = {
    "id": uuid.uuid4(),
    "name": fake.name(),
    "rural": True,
    "underrepresented": True,
    "timezone": random.randint(-8, 4),
    "interestCompanies": ['Davis, Sosa and Wall', fake.company(), fake.company(), fake.company()],
    "interestTags": fake.words(),
    "requireExtended": False
}


In [18]:
# background_rural = {
#     "term": {
#         "backgroundRural": {
#             "value": True
#         }
#     }
# }
#
# query_body = {
#     "query": {
#         background_rural
#     }
# }
#
# result = client.search(index="mentors_index", body=query_body)
# hits = result["hits"]["hits"]


In [20]:
from elasticsearch_dsl import Q
from elasticsearch_dsl.query import MatchNone, MatchAll

s = elasticsearch_dsl.Search(using=client, index='mentors_index').highlight("backgroundRural").extra(explain=True)

# Adds one to all quer
base_value = Q("constant_score", filter=MatchAll())

# Uses a fuzzy query to determine if a person works for a company in the student's list, then if so adds to the score
company_q = None
for company in student["interestCompanies"]:
    if company_q is None:
        company_q = Q("function_score", query=Q("fuzzy", company=company), weight="10", score_mode="multiply")
    else:
        company_q = (company_q | Q("function_score", query=Q("fuzzy", company=company), weight="10", score_mode="multiply"))

# If background_rural matches on mentor and student, then adds one to the query
background_rural = Q("constant_score", filter=Q("term", backgroundRural=student['rural']))

# If student is underrepresented, adds the value of prefer_student_underrep to the query
if student["underrepresented"]:
    prefer_student_underrep = Q({
        "function_score": {
            "field_value_factor" : {
                "field": "preferStudentUnderRep",
                "factor": 1,
                "modifier": "none",
                "missing": 0
            }
        }
    })
else:
    # Adds 0 to query if nothing is found
    prefer_student_underrep = Q("constant_score", filter=MatchNone())

if student["requireExtended"]:
    requireExtended = Q("term", okExtended=True)
    s = s.query((base_value | company_q | background_rural | prefer_student_underrep) & requireExtended)
else:
    s = s.query(base_value | company_q | background_rural | prefer_student_underrep)

resp = s.execute()
hits = resp["hits"]
print(hits)

ValueError: dictionary update sequence element #0 has length 5; 2 is required